# Assignment 1: Transformations and Representations

Roll number: 2019101068


## 2. Euler Angles, Rotation Matrices, and Quaternions
1. Write a function (do not use inbuilt libraries for this question):
    - that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians **(X-Y-Z)**.
    - to convert a rotation matrix to quaternion and vice versa. 

2. What is a Gimbal lock? Suppose an airplane increases its pitch from $0°$ to $90°$. 

    - Let $R_{gmb\beta}$ be the rotation matrix for $\beta=90°$. Find $R_{gmb\beta}$.
    - Consider the point $p = [0, 1, 0]ᵀ $ on the pitched airplane, i.e. the tip of the wing. Does there exist any $α , γ$ such that $p_{new} = R_{gmb\beta}\; p$ for:
      1. $p_{new} = [1, 0, 0]ᵀ $
      2. For some  $p_{new}$ on the XY unit circle?
      3. For some  $p_{new}$ on the YZ unit circle?
      
      Show your work for all three parts and briefly explain your reasoning. Why is $\beta=90°$  a “certain problematic value”?

    <img src="img/2.3.jpeg"  width="500" ><br>
    
    <img src="img/2.1.jpeg"  width="500" ><br>

    <img src="img/2.2.jpeg"  width="500" >
    


## Rotation matrix from euler angles

In [1]:
import numpy as np
import math


## Here, I am following the convention given in book by Craig: Section 2.8

#### On Moodle it has been mentioned: You can assume alpha, beta and gamma angles (rotation magnitude) in radian/degree to be about Z, Y, X axis respectively.

Since nothing has been mentioned regarding extrinsic/intrinsic ie extrinsic (rotations about the axes xyz of the original coordinate system, which is assumed to remain motionless) or intrinsic (rotations about the axes of the rotating coordinate system XYZ, solidary with the moving body, which changes its orientation after each elemental rotation), I am assuming **Intrinsic** rotation here

### NOTE: the given function calculates rotation as per:
* **X–Y–Z fixed angles (extrinsic)**
* **Z-Y-X rotating angles (intrinsic)**

The result comes out to be the same in both cases.

**See Chapter 2 of Craig book for convention**

In [2]:
def euler_angles_dict_to_rotation_matrix( angles_dict):
    angle_x = angles_dict["gamma"]
    angle_y = angles_dict["beta"]
    angle_z = angles_dict["alpha"]

    rot_x = np.array( [ [ 1 , 0 , 0 ] ,
                           [ 0 , math.cos(angle_x) , -math.sin(angle_x) ]  , 
                           [0 , math.sin(angle_x) , math.cos(angle_x)]])

    rot_y = np.array( [ [ math.cos(angle_y) , 0 , math.sin(angle_y) ] ,
                           [ 0 ,1 ,0 ] ,
                           [ -math.sin(angle_y) , 0 , math.cos(angle_y)]])
    
    
    rot_z = np.array( [ [  math.cos(angle_z) , -math.sin(angle_z) , 0 ] , 
                               [math.sin(angle_z) , math.cos(angle_z) , 0 ] ,
                               [0 ,0, 1]]  )
    
    FINAL_R = rot_z@rot_y@rot_x

    return FINAL_R

In [3]:
# make changes here
angles = dict({
    "alpha": math.radians(12), 
    "beta": math.radians(34),
    "gamma": math.radians(23)
})
ROT_MAT = euler_angles_dict_to_rotation_matrix(angles)
print(ROT_MAT)

[[ 0.81092111  0.02233573  0.58472905]
 [ 0.1723666   0.94581709 -0.2751723 ]
 [-0.5591929   0.32393079  0.76313311]]


## Quat to Rot Matrix

### Here, I am assuming q = (magnitude, vector) ie (w, x, y, z) format

#### Derived formula reverified from Craig book pg 50: 
Tested using this: https://www.andre-gaschler.com/rotationconverter/ 

In [4]:
def quat_to_rot(q):
    
    # normalize quaternions
    q = q / np.sqrt(np.sum(q**2))
    
    # get components
    w, x, y, z = q
    
    ans = np.zeros((3,3))
    
    
    ans[0,0]=1 - 2*(y*y + z * z)
    ans[0,1] = 2*(-z*w + y*x)
    ans[0,2] = 2*(y*w + z*x)

    ans[1,0] = 2*(x*y + w*z)
    ans[1,1] = 1 - 2*(z*z + x*x)
    ans[1,2] =   2*(y*z - x*w)

    ans[2,0] = 2*(x*z - w*y)
    ans[2,1] = 2*(y*z + w*x)
    ans[2,2] = 1 - 2*(y*y + x*x)
    
    # the convention has not been mentioned whether it is from frame F1 to F2 or vice versa
    # If reverse is desired, please take transpose
    #ans=ans.T
                            
    return ans

In [5]:
q = np.array([0.1, 0.462, 0.191, -0.462])
q = np.array([1, 1, 3, 121])

In [6]:
R = quat_to_rot(q)
print(R)

[[-0.999727   -0.01610702  0.01692602]
 [ 0.01692602 -0.998635    0.04941305]
 [ 0.01610702  0.04968605  0.998635  ]]


### Rotation mat to quaternions

##### Handle square root of negative trace:: DONE

##### Verified derivation: http://www.euclideanspace.com/maths/geometry/rotations/conversions/matrixToQuaternion/

In [7]:
def rotation_to_quat(R):
      
    mat_trace = R[0][0] + R[1][1] + R[2][2]
    if(mat_trace > 0):
        tmp = math.sqrt(1 + mat_trace)*2
        w = 0.25*tmp
        x = (R[2][1] - R[1][2])/tmp
        y = (R[0][2] - R[2][0])/tmp
        z = (R[1][0] - R[0][1])/tmp
    # pick path corresponding to largest of x and y and z
    elif(R[0][0] > R[1][1] and R[0][0] > R[2][2]):
        tmp = math.sqrt(1 + R[0][0] - R[1][1] - R[2][2])*2
        w = (R[2][1] - R[1][2])/tmp
        x = 0.25*tmp
        y = (R[0][1] + R[1][0])/tmp
        z = (R[0][2] + R[2][0])/tmp
    # largest has to be among R[1][1] and R[1][2]
    elif(R[1][1] > R[2][2]):
        tmp = math.sqrt(1 + R[1][1] - R[0][0] - R[2][2])*2
        w = (R[0][2] - R[2][0])/tmp
        x = (R[0][1] + R[1][0])/tmp
        y = 0.25*tmp
        z = (R[1][2] + R[2][1])/tmp
    else:
        tmp = math.sqrt(1 + R[2][2] - R[0][0] - R[1][1])*2
        w = (R[1][0] - R[0][1])/tmp
        x = (R[0][2] + R[2][0])/tmp
        y = (R[1][2] + R[2][1])/tmp
        z = 0.25*tmp

    quat = np.array([w, x,y,z])

    # already normalized before this
    quat = quat / math.sqrt(np.sum(quat**2))
    return quat

In [8]:
R = np.array([ [-0.8562221,  0.2498761,  0.4521567],
   [0.3212692, -0.4278632,  0.8448190],
   [0.4045612,  0.8686168,  0.2860684 ]])

In [9]:
q=rotation_to_quat(R)

In [10]:
q

array([0.02226623, 0.26719498, 0.53438996, 0.80158494])

## FOR PART 2, please see pdf